In [ ]:
import keras
import numpy as np
from keras.applications import vgg19
from keras.applications.vgg19 import preprocess_input
model = vgg19.VGG19(weights='imagenet', include_top=True)

In [ ]:
model.summary()

In [ ]:
image_path='/Users/Santiago/Documents/Database/davis-data/JPEGImages/480p/bear/00000.jpg'
image_path='/Users/Santiago/Documents/Database/davis-data/JPEGImages/480p/boat/00000.jpg'
image_path='/Users/Santiago/Documents/Database/davis-data/JPEGImages/480p/camel/00000.jpg'
#image_path='/Users/Santiago/Documents/Database/davis-data/JPEGImages/480p/blackswan/00000.jpg'

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def decode_vgg(x):
    xin=np.copy(x[:,:,:])
    xin=xin.astype('float32')
    xin[:,:,0]+=103.939
    xin[:,:,1]+=116.779
    xin[:,:,2]+=123.68
    return xin[:, :, ::-1]

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
img = load_img(image_path,target_size=(224, 224))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x=x.astype('float32')
x = preprocess_input(x)
pred_class = model.predict(np.array(x))
dec_pred=vgg19.decode_predictions(pred_class)
#print(pred_class.shape)
num_classes=pred_class.shape[1]
plt.plot(pred_class[0,:],'.')
plt.xlabel('Classes')
plt.ylabel('Prediction')
plt.title(dec_pred[0][0][1:])
plt.show()
pred_class=np.argmax(pred_class)


In [ ]:
from keras import losses
import keras.backend as K
input_class = K.zeros(shape=(1,num_classes))
loss = losses.categorical_crossentropy(input_class, model.output)
im_input = model.input # This is the input image
grads = K.gradients(loss, im_input)[0] # the signs will acheive the desired effect
grads /= K.maximum(K.mean(K.abs(grads)), K.epsilon()) # Normalize for numerical stability
outputs = [grads]
adv_noise = K.function([im_input, input_class], outputs)

In [ ]:
xiter=x
pred_class = model.predict(np.reshape(np.array(x),[1,224,224,3]))
pred_class=np.argmax(pred_class)
label=np.zeros(num_classes)
label[pred_class]=1

In [ ]:
S=adv_noise([xiter,np.expand_dims(label,0)])[0]
plt.figure(figsize=(10,3))
plt.subplot(1,3,1)
plt.imshow(S[0,:,:,0])
plt.subplot(132)
plt.imshow(S[0,:,:,1])
plt.subplot(133)
plt.imshow(S[0,:,:,2])
plt.show()

print('Range of Adversarial Noise')
print(S.max())
print(S.min())





In [ ]:
for c in range(5):
    Xadv=x+(c/10)*(S)
    pred_class_adv = model.predict(Xadv)
    dec_pred=vgg19.decode_predictions(pred_class_adv)
    plt.plot(pred_class_adv[0,:],'.')
    plt.xlabel('Classes')
    plt.ylabel('Prediction')
    plt.title(dec_pred[0][0][1:])
    plt.show()
    #pred_class_adv=np.argmax(pred_class_adv)
    plt.imshow(decode_vgg((Xadv)[0,:,:,:])/255)
    plt.show()